In [ ]:
import os
import subprocess
import time
import json
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"  # ✅ Uses the same JSON file
TOKEN_FILE = "token.json"  # ✅ Stores OAuth token here

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    Reuses saved credentials to avoid repeated logins.
    """
    creds = None

    # ✅ Load existing credentials if available
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    # ✅ Refresh expired credentials or request new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # ✅ Save credentials to avoid future logins
        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

def merge_videos_and_create_timestamps(folder_path):
    """
    Converts MP4 videos to a uniform format, merges them, and creates timestamps.
    Returns the merged video path and description content (timestamps).
    """
    if not os.path.exists(folder_path):
        print(f"❌ Error: The folder path '{folder_path}' does not exist.")
        return None, None

    # Gather video files
    video_files = []
    timestamps = []
    total_duration = 0
    converted_files = []

    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(".mp4"):
            filepath = os.path.join(folder_path, filename)
            video_files.append(filepath)

            try:
                probe = subprocess.run(
                    ["ffprobe", "-v", "error", "-show_entries", "format=duration",
                     "-of", "default=noprint_wrappers=1:nokey=1", filepath],
                    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
                )
                duration = float(probe.stdout.strip())
                timestamps.append((filename, total_duration))
                total_duration += duration
            except Exception as e:
                print(f"⚠️ Warning: Error reading duration for '{filename}': {e}")

    if not video_files:
        print("❌ No MP4 files found.")
        return None, None

    # Step 1: Convert All Videos to a Standard Format
    print("🔄 Converting videos to a uniform format (H.264, AAC)...")
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)

    for idx, video in enumerate(video_files):
        converted_video = os.path.join(converted_folder, f"converted_{idx}.mp4")
        subprocess.run(
            ["ffmpeg", "-i", video, "-c:v", "libx264", "-c:a", "aac", "-strict", "experimental", converted_video],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        converted_files.append(converted_video)

    # Step 2: Prepare the file list for FFmpeg
    file_list_path = os.path.join(folder_path, "file_list.txt")
    with open(file_list_path, "w") as f:
        for converted_file in converted_files:
            f.write(f"file '{converted_file}'\n")

    # Step 3: Merge videos using FFmpeg
    merged_video_path = os.path.join(folder_path, "merged_video.mp4")
    try:
        print("🔗 Merging videos...")
        subprocess.run(
            ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list_path, "-c", "copy", merged_video_path],
            check=True
        )
    except subprocess.CalledProcessError as e:
        print(f"❌ Error during merging: {e}")
        return None, None

    # Step 4: Generate timestamps in HH:MM:SS format
    timestamp_file_path = os.path.join(folder_path, "timestamps.txt")
    with open(timestamp_file_path, "w") as f:
        description = ""
        for filename, timestamp in timestamps:
            hours = int(timestamp // 3600)
            minutes = int((timestamp % 3600) // 60)
            seconds = int(timestamp % 60)
            timestamp_entry = f"{filename} - {hours:02d}:{minutes:02d}:{seconds:02d}"
            f.write(timestamp_entry + "\n")
            description += timestamp_entry + "\n"

    print(f"✅ Merged video saved to: {merged_video_path}")
    print(f"✅ Timestamps saved to: {timestamp_file_path}")

    return merged_video_path, description


def upload_to_youtube(video_path, title, description, folder_path):
    """
    Uploads a video to YouTube and saves the video URL in 'youtube link.txt'.
    """
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": ["training", "tutorial", "video"],
                "categoryId": "27"
            },
            "status": {
                "privacyStatus": "unlisted"
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    print("Uploading video to YouTube...")
    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%")
        except Exception as e:
            raise Exception(f"Error during upload: {e}")

    video_id = response['id']
    youtube_url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"✅ Video uploaded successfully! Video URL: {youtube_url}")

    # ✅ Save the video URL to "youtube link.txt" inside the same folder
    link_file_path = os.path.join(folder_path, "youtube link.txt")
    with open(link_file_path, "w") as f:
        f.write(youtube_url + "\n")

    print(f"✅ YouTube link saved in: {link_file_path}")

def upload_to_youtube_with_retry(video_path, title, description, folder_path, retries=3, delay=10):
    """
    Uploads a video to YouTube with retry logic.
    """
    for attempt in range(retries):
        try:
            upload_to_youtube(video_path, title, description, folder_path)
            return
        except Exception as e:
            print(f"Error during upload: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("❌ Upload failed after multiple attempts.")

if __name__ == "__main__":
    folder_path = input("Enter the folder path containing the videos: ").strip()

    # Step 1: Merge videos and create timestamps
    merged_video_path, description = merge_videos_and_create_timestamps(folder_path)

    # Step 2: Upload merged video to YouTube with retry logic
    if merged_video_path and description:
        title = os.path.basename(merged_video_path).replace(".mp4", "")  # Use the merged video name as the title
        upload_to_youtube_with_retry(merged_video_path, title, description, folder_path)


🔄 Converting videos to a uniform format (H.264, AAC)...
🔗 Merging videos...
✅ Merged video saved to: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\InDesign\Lynda\InDesign CC 2018 - New Features\merged_video.mp4
✅ Timestamps saved to: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\InDesign\Lynda\InDesign CC 2018 - New Features\timestamps.txt
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62304%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=jCfSc8BSggFWYqBB3jOa9PUa8wqEAe&access_type=offline
Uploading video to YouTube...
✅ Video uploaded successfully! Video URL: https://www.youtube.com/watch?v=Fn83wlZrzzo
✅ YouTube link saved in: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\InDesign\Lynda\InDesign CC 2018 - New Features\youtube link.txt
